# Fetch song list

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("notshrirang/spotify-million-song-dataset")

In [3]:
import os

# List all files in the dataset directory
for root, dirs, files in os.walk(path):
    for file in files:
        print(os.path.join(root, file))

C:\Users\celin\.cache\kagglehub\datasets\notshrirang\spotify-million-song-dataset\versions\1\spotify_millsongdata.csv


In [207]:
import pandas as pd

# Set the full path to the CSV
file_path = r"C:\Users\celin\.cache\kagglehub\datasets\notshrirang\spotify-million-song-dataset\versions\1\spotify_millsongdata.csv"

# Load the dataset
df = pd.read_csv(file_path)

# Show the first 5 rows
display(df.head())

#lenght of dataset
print(len(df))

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


57650


In [208]:
df = df.drop("link", axis=1)

In [209]:
df = df.head(1000).copy()  # random_state makes it reproducible
df

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
995,Backstreet Boys,Satellite,Here I am again \r\nWaiting on the moment you...
996,Backstreet Boys,She's A Dream,Oh \r\nOh \r\nOh oh oh yeah \r\nOh no \r\n...
997,Backstreet Boys,Shining Star,[Chorus] \r\nYou're my shining star \r\nThat...
998,Backstreet Boys,Shout,Ignorance of people purchasing diamonds and ne...


# Clean text

In [6]:
...

Ellipsis

# Search for attributes

#### Access to API's

In [7]:
import musicbrainzngs
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# ==== SPOTIFY API SETUP ====
SPOTIFY_CLIENT_ID = "0aedb24eab69401ca05adb1392506687"
SPOTIFY_CLIENT_SECRET = "1424f01574054adab515e44bcdb36141"

sp_auth = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
spotify = spotipy.Spotify(auth_manager=sp_auth)

# ==== MUSICBRAINZ SETUP ====
musicbrainzngs.set_useragent("ComputationalSocialProject", "1.0", "s234809@dtu.dk")

In [45]:
def get_song_metadata(title, artist):
    data = {
        "artist": artist,
        "title": title,
        "writers": [],
        "genres": [],
        "popularity": None,
        "release_year": None  # <-- New field
    }

    # --- MusicBrainz: Writers ---
    try:
        work_result = musicbrainzngs.search_works(query=title, artist=artist, limit=1)
        if work_result['work-list']:
            work_id = work_result['work-list'][0]['id']
            work_data = musicbrainzngs.get_work_by_id(work_id, includes=["artist-rels"])
            for rel in work_data['work'].get("artist-relation-list", []):
                if rel['type'] in ["composer", "lyricist", "writer"]:
                    data["writers"].append(f"{rel['artist']['name']}")
    except Exception as e:
        print(f"[MusicBrainz Writers Error] {e}")

    # --- Spotify: Popularity + Genre from Artist + Release Year ---
    try:
        result = spotify.search(q=f"track:{title} artist:{artist}", type="track", limit=1)
        if result['tracks']['items']:
            track = result['tracks']['items'][0]
            data["popularity"] = track["popularity"]

            # Get genres from main artist
            artist_id = track['artists'][0]['id']
            artist_info = spotify.artist(artist_id)
            data["genres"] = artist_info.get("genres", [])

            # Get release year from album
            release_date = track['album']['release_date']
            if release_date:
                data["release_year"] = release_date.split("-")[0]  # Extract year part
    except Exception as e:
        print(f"[Spotify Error] {e}")

    return data


In [147]:
# Add empty columns to hold metadata
df["writers"] = df.get("writers", pd.Series(dtype='object'))
df["genres"] = df.get("genres", pd.Series(dtype='object'))
df["popularity"] = df.get("popularity", pd.Series(dtype='object'))
df["release_year"] = df.get("release_year", pd.Series(dtype='object'))


In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
    meta = get_song_metadata(df.iloc[i]["song"], df.iloc[i]["artist"])
    
    df.loc[i, "writers"] = ", ".join(meta["writers"]) if isinstance(meta["writers"], list) else meta["writers"]
    df.loc[i, "genres"] = ", ".join(meta["genres"]) if isinstance(meta["genres"], list) else meta["genres"]
    df.loc[i, "popularity"] = meta["popularity"]
    df.loc[i, "release_year"] = meta["release_year"]

In [ ]:
def fetch_metadata(i):    
    try:
        meta = get_song_metadata(df.iloc[i]["song"], df.iloc[i]["artist"])
        return {
            "index": int(i),
            "writers": ", ".join(meta["writers"]),
            "genres": ", ".join(meta["genres"]),
            "popularity": meta["popularity"],
            "release_year": meta["release_year"]
        }
    except Exception as e:
        print(f"[Error at index {i}]: {e}")
        return None

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

# Run metadata fetching in parallel
results = Parallel(n_jobs=-1, backend="threading")(
    delayed(fetch_metadata)(i) for i in tqdm(range(len(df)))
)

# Filter out bad entries
valid_results = [r for r in results if r is not None]

# Update DataFrame after parallel fetch
for r in results:
    df.loc[r["index"], "writers"] = r["writers"]
    df.loc[r["index"], "genres"] = r["genres"]
    df.loc[r["index"], "popularity"] = r["popularity"]
    df.loc[r["index"], "release_year"] = r["release_year"]


In [191]:
df

,artist,song,text,genres,popularity,release_year,song_spotify,artist_spotify,genres_spotify,popularity_spotify,...,genres_spotify,popularity_spotify,release_year_spotify,song_spotify,artist_spotify,genres_spotify,popularity_spotify,release_year_spotify,song_spotify,artist_spotify
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",,59.0,1980,"Andante, Andante",ABBA,,59.0,...,,59.0,1980,"Andante, Andante",ABBA,,59.0,1980,"Andante, Andante",ABBA
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,,49.0,1979,As Good As New,ABBA,,49.0,...,,49.0,1979,As Good As New,ABBA,,49.0,1979,As Good As New,ABBA
3,ABBA,Bang,Making somebody happy is a question of give an...,,51.0,1975,Bang,ABBA,,51.0,...,,51.0,1975,Bang,ABBA,,51.0,1975,Bang,ABBA
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,,51.0,1975,Bang-A-Boomerang,ABBA,,51.0,...,,51.0,1975,Bang-A-Boomerang,ABBA,,51.0,1975,Bang-A-Boomerang,ABBA
5,ABBA,Burning My Bridges,"Well, you hoot and you holler and you make me ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ABBA,Cassandra,Down in the street they're all singing and sho...,,40.0,1981,Cassandra,ABBA,,40.0,...,,40.0,1981,Cassandra,ABBA,,40.0,1981,Cassandra,ABBA
7,ABBA,Chiquitita,"Chiquitita, tell me what's wrong \r\nYou're e...",,76.0,1979,Chiquitita,ABBA,,76.0,...,,76.0,1979,Chiquitita,ABBA,,76.0,1979,Chiquitita,ABBA
8,ABBA,Crazy World,I was out with the morning sun \r\nCouldn't s...,,37.0,1975,Crazy World,ABBA,,37.0,...,,37.0,1975,Crazy World,ABBA,,37.0,1975,Crazy World,ABBA
9,ABBA,Crying Over You,I'm waitin' for you baby \r\nI'm sitting all ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Iteration 2

In [210]:
batch_size = 20
rows = df.reset_index()[["index", "song", "artist"]].values.tolist()
batches = [rows[i:i + batch_size] for i in range(0, len(rows), batch_size)]

def fetch_spotify_batch(batch):
    results = []
    for idx, title, artist in batch:
        try:
            result = spotify.search(q=f"track:{title} artist:{artist}", type="track", limit=1)
            if result['tracks']['items']:
                track = result['tracks']['items'][0]
                artist_id = track['artists'][0]['id']
                artist_info = spotify.artist(artist_id)

                results.append({
                    "index": idx,
                    "genres": ", ".join(artist_info.get("genres", [])),
                    "popularity": track["popularity"],
                    "release_year": track["album"]["release_date"].split("-")[0],
                })
        except Exception as e:
            print(f"[Spotify Error @ {title} - {artist}]: {e}")
    return results

spotify_results = Parallel(n_jobs=-1)(
    delayed(fetch_spotify_batch)(batch) for batch in tqdm(batches, desc="Spotify metadata")
)
flat_spotify = [item for sublist in spotify_results for item in sublist]
spotify_df = pd.DataFrame(flat_spotify).set_index("index")


df = df.merge(spotify_df, how="left", left_index=True, right_index=True)


















Spotify metadata: 100%|██████████| 50/50 [00:43<00:00,  1.15it/s]


In [211]:
df

,artist,song,text,genres,popularity,release_year
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",NaN,NaN,NaN
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",,59.0,1980
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,,49.0,1979
3,ABBA,Bang,Making somebody happy is a question of give an...,,51.0,1975
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,,51.0,1975
...,...,...,...,...,...,...
995,Backstreet Boys,Satellite,Here I am again \r\nWaiting on the moment you...,NaN,NaN,NaN
996,Backstreet Boys,She's A Dream,Oh \r\nOh \r\nOh oh oh yeah \r\nOh no \r\n...,,30.0,2009
997,Backstreet Boys,Shining Star,[Chorus] \r\nYou're my shining star \r\nThat...,,41.0,2000
998,Backstreet Boys,Shout,Ignorance of people purchasing diamonds and ne...,NaN,NaN,NaN


In [213]:
df = df.drop("genres", axis=1)


In [214]:
df = df.dropna()

In [215]:
df

,artist,song,text,popularity,release_year
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",59.0,1980
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...,49.0,1979
3,ABBA,Bang,Making somebody happy is a question of give an...,51.0,1975
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...,51.0,1975
6,ABBA,Cassandra,Down in the street they're all singing and sho...,40.0,1981
...,...,...,...,...,...
992,Backstreet Boys,Roll With It,Roll with it \r\nRoll with it \r\nRoll with ...,33.0,1996
994,Backstreet Boys,Safest Place To Hide,"Yeah, yeah \r\nIt seems like yesterday when I...",38.0,2005
996,Backstreet Boys,She's A Dream,Oh \r\nOh \r\nOh oh oh yeah \r\nOh no \r\n...,30.0,2009
997,Backstreet Boys,Shining Star,[Chorus] \r\nYou're my shining star \r\nThat...,41.0,2000


In [220]:
import numpy as np
present_years = df["release_year"].dropna().astype(int).unique().tolist()
np.sort(present_years)

array([1957, 1959, 1963, 1964, 1967, 1968, 1969, 1970, 1971, 1972, 1973,
       1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984,
       1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2017, 2018,
       2019, 2020, 2021, 2022, 2023, 2024, 2025])